In [ ]:
###Combining the DataSets to get the Final CSV File

In [ ]:
import pandas as pd
##Read file  (Transactional Data) as dataframe
orders = pd.read_csv("orders.csv")
#Read users (User Master Data) as dataframe
users=pd.read_json("users.json")




In [28]:
## Restaurant Master Data
sql_file_path = "restaurants.sql"


In [29]:
##Connecting sql ith sqlite3
import sqlite3
import pandas as pd


In [ ]:
##Providing the connecton to the database and creates DB
conn=sqlite3.connect("restaurants.db")  
cursor=conn.cursor()


In [31]:
##File handling
with open("restaurants.sql","r",encoding="utf-8") as file:
    sql_script = file.read() #Reads the file ans store as string


In [ ]:
cursor.executescript(sql_script) #Excutes the sql cmds
conn.commit()  #saves the changes


In [33]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)


,name
0,restaurants


In [34]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [35]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


In [36]:
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)


In [ ]:
##Final to CSV file conversion

final_df.to_csv("final_food_delivery_dataset.csv", index=False)
print(final_df)


      order_id  user_id  restaurant_id  order_date  total_amount  \
0            1     2508            450  18-02-2023        842.97   
1            2     2693            309  18-01-2023        546.68   
2            3     2084            107  15-07-2023        163.93   
3            4      319            224  04-10-2023       1155.97   
4            5     1064            293  25-12-2023       1321.91   
...        ...      ...            ...         ...           ...   
9995      9996     2528            249  21-05-2023       1211.96   
9996      9997     2867            267  06-08-2023       1188.05   
9997      9998      522            420  11-11-2023        979.44   
9998      9999      319            492  08-09-2023       1105.93   
9999     10000      457            439  21-10-2023        879.58   

                   restaurant_name_x       name       city membership  \
0                  New Foods Chinese  User_2508  Hyderabad    Regular   
1     Ruchi Curry House Multicuisine 

In [ ]:
### MCQ Questions

In [1]:
import pandas as pd

In [ ]:
#highest total revenue (total_amount) from Gold members
df=pd.read_csv("final_food_delivery_dataset.csv")
gold_df=df[df["membership"]=="Gold"]
city_revenue=(
  gold_df.groupby("city")["total_amount"].sum().sort_values(ascending=False)
)
city_revenue

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [ ]:
#cuisine has the highest average order value across all orders
cuisine_avg = (
    df.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)
cuisine_avg

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [ ]:
user_spend = df.groupby("user_id")["total_amount"].sum()

# Users with total spend
#  > 1000
count_users = (user_spend > 1000).sum()

count_users

2544

In [6]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)

rating_revenue = (
    df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue

C:\Users\Rashu\AppData\Local\Temp\ipykernel_13252\2692918571.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]


rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [ ]:
# the highest average order value,Gold numbers
gold_df = df[df["membership"] == "Gold"]

avg_order_value_city = (
    gold_df.groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

avg_order_value_city

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [ ]:
#the lowest number of distinct restaurants but still contributes significant revenue
restaurant_count = df.groupby("cuisine")["restaurant_id"].nunique()
revenue = df.groupby("cuisine")["total_amount"].sum()

summary = pd.DataFrame({
    "distinct_restaurants": restaurant_count,
    "total_revenue": revenue
}).sort_values("distinct_restaurants")

summary

,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [ ]:
# total orders were placed by Gold members
total_orders = len(df)
gold_orders = len(df[df["membership"] == "Gold"])

percentage = round((gold_orders / total_orders) * 100)

percentage

50

In [10]:
restaurant_stats = (
    df.groupby("restaurant_name_x")
    .agg(
        total_orders=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
)

# Filter restaurants with < 20 orders
filtered = restaurant_stats[restaurant_stats["total_orders"] < 20]

# Given options
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

filtered.loc[filtered.index.intersection(options)]

,total_orders,avg_order_value
restaurant_name_x,,
Ruchi Foods Chinese,19,686.603158


In [11]:
combos = {
    "Gold + Indian cuisine": (df["membership"] == "Gold") & (df["cuisine"] == "Indian"),
    "Gold + Italian cuisine": (df["membership"] == "Gold") & (df["cuisine"] == "Italian"),
    "Regular + Indian cuisine": (df["membership"] == "Regular") & (df["cuisine"] == "Indian"),
    "Regular + Chinese cuisine": (df["membership"] == "Regular") & (df["cuisine"] == "Chinese"),
}

revenue = {k: df[v]["total_amount"].sum() for k, v in combos.items()}
revenue

{'Gold + Indian cuisine': 979312.31,
 'Gold + Italian cuisine': 1005779.05,
 'Regular + Indian cuisine': 992100.27,
 'Regular + Chinese cuisine': 952790.9099999999}

In [15]:
print(df.columns)


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'quarter'],
      dtype='object')


In [19]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")



df['order_date'] = pd.to_datetime(df['order_date'])
df['quarter'] = df['order_date'].dt.to_period('Q')

# replace 'amount' with the correct column name after seeing df.columns
quarterly_revenue = df.groupby('quarter')['total_amount'].sum()

print("\nQuarter-wise Revenue:")
print(quarterly_revenue)

print("\nCorrect Option:")
print(quarterly_revenue.idxmax())



Quarter-wise Revenue:
quarter
2023Q1    1993425.14
2023Q2    1945348.72
2023Q3    2037385.10
2023Q4    2018263.66
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

Correct Option:
2023Q3


C:\Users\Rashu\AppData\Local\Temp\ipykernel_13252\3995408946.py:7: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


In [ ]:
##Numerical answers

In [ ]:
#total orders were placed by users with Gold membership
gold_orders = df[df["membership"] == "Gold"]
len(gold_orders)

4987

In [ ]:
# total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city
hyderabad_revenue = df[df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyderabad_revenue)

1889367

In [ ]:
#distinct users placed at least one order
df = pd.read_csv("final_food_delivery_dataset.csv")
df["user_id"].nunique()

2883

In [ ]:
#the average order value (rounded to 2 decimals) for Gold members?
# Filter Gold members
gold_orders = df[df['membership'] == 'Gold']

# Calculate average order value
avg_order_value = round(gold_orders['total_amount'].mean(), 2)

avg_order_value

797.15

In [25]:
import pandas as pd

# Load final dataset
df = pd.read_csv('final_food_delivery_dataset.csv')

# Filter orders with restaurant rating >= 4.5
high_rating_orders = df[df['rating'] >= 4.5]

# Count total orders
order_count = high_rating_orders.shape[0]

print(order_count)


3374


In [26]:
import pandas as pd

# Load dataset
df = pd.read_csv('final_food_delivery_dataset.csv')

# Step 1: Filter Gold members
gold_df = df[df['membership'] == 'Gold']

# Step 2: Find top revenue city among Gold members
city_revenue = gold_df.groupby('city')['total_amount'].sum()
top_city = city_revenue.idxmax()

# Step 3: Count orders in that city
order_count = gold_df[gold_df['city'] == top_city].shape[0]

print(top_city, order_count)


Chennai 1337
